In [ ]:
x = torch.rand(4,6,5) #batch_size*feature_dim*num_points

###
#caculate the distance affinity matrix: 
inner = -2*torch.matmul(x.transpose(2, 1), x)
xx = torch.sum(x**2, dim=1, keepdim=True)
pairwise_distance = -xx - inner - xx.transpose(2, 1) #batch_size*num_points*num_points
###

###
#使用热核函数构建weight matrix and compute the laplace matrix：
d = np.square(pairwise_distance)
d = d/2 # 2 is t
w = -torch.exp(d)
b = torch.sum(w,dim=1)
b = b.reshape(4,5,1).repeat(1,1,5)
c = torch.eye(5,5).expand(4,5,5)
D = b*c
L = D-w
###

###
#compute the eigen of laplace:
lamda,f = np.linalg.eig(L)
###

###
#最小的m个非零特征值对应的特征向量作为降维后的结果输出,这里 m=3
inds = np.argsort(lamda,axis=1)
inds = inds[:,0:3] #3 is the dim of feature we need

base = (np.arange(0,4))*5
base = base.reshape(4,1)
inds = inds+base
inds = inds.reshape(4*3)

f=torch.from_numpy(f)
f = torch.transpose(f, 1, -1)

f=f.reshape(20,5) #(batch_size*num_points)*num_points

f = f[inds,:]

f=f.reshape(4,3,5)
f = torch.transpose(f, 1, -1) #final result: batch_size*num_points*feature we need